# DeepShot: Data Standardization

Standardizing team names, player information, and coordinate systems across datasets.

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set up directories
data_dir = Path('../data')
raw_dir = data_dir / 'raw'
interim_dir = data_dir / 'interim'
processed_dir = data_dir / 'processed'
processed_dir.mkdir(parents=True, exist_ok=True)

In [95]:
# Load cleaned data
shots_path = interim_dir / 'shots_clean.csv'
team_stats_path = interim_dir / 'team_stats_clean.csv'

if shots_path.exists():
    shots = pd.read_csv(shots_path)
    print(f"Loaded {len(shots)} shot records")
else:
    raise FileNotFoundError(f"Required data file not found: {shots_path}")

if team_stats_path.exists():
    team_stats = pd.read_csv(team_stats_path)
    print(f"Loaded {len(team_stats)} team stat records")
else:
    raise FileNotFoundError(f"Required data file not found: {shots_path}")

player_data_path = interim_dir / 'player_data_clean.csv'
if player_data_path.exists():
    player_data = pd.read_csv(player_data_path)
    
    # Rename 'tm' to 'team' for consistency
    if 'tm' in player_data.columns:
        player_data.rename(columns={'tm': 'team'}, inplace=True)
        print("Renamed 'tm' column to 'team' in player data for consistency")
        
    print(f"Loaded {len(player_data)} player records")
else:
    raise FileNotFoundError(f"Required data file not found: {player_data_path}")

Loaded 4231070 shot records
Loaded 1876 team stat records
Renamed 'tm' column to 'team' in player data for consistency
Loaded 32538 player records


## Team Name Standardization

In [96]:
# Create team name mapping
team_mapping = {
    # Washington Wizards and historical names
    'Washington': 'Washington Wizards',
    'WAS': 'Washington Wizards',
    'Wizards': 'Washington Wizards',
    'Washington Wizards': 'Washington Wizards',
    'Bullets': 'Washington Wizards',
    'Washington Bullets': 'Washington Wizards',
    'Capital Bullets': 'Washington Wizards',
    'Baltimore Bullets': 'Washington Wizards',
    'Chicago Zephyrs': 'Washington Wizards',
    'Chicago Packers': 'Washington Wizards',
    
    # Atlanta Hawks and historical names
    'Atlanta': 'Atlanta Hawks',
    'ATL': 'Atlanta Hawks',
    'Hawks': 'Atlanta Hawks',
    'Atlanta Hawks': 'Atlanta Hawks',
    'St. Louis Hawks': 'Atlanta Hawks',
    'Milwaukee Hawks': 'Atlanta Hawks',
    'Tri-Cities Blackhawks': 'Atlanta Hawks',
    
    # Los Angeles Clippers and historical names
    'LA Clippers': 'Los Angeles Clippers',
    'LAC': 'Los Angeles Clippers',
    'Clippers': 'Los Angeles Clippers',
    'Los Angeles Clippers': 'Los Angeles Clippers',
    'Buffalo Braves': 'Los Angeles Clippers',
    'San Diego Clippers': 'Los Angeles Clippers',
    
    # Sacramento Kings and historical names
    'Sacramento': 'Sacramento Kings',
    'SAC': 'Sacramento Kings',
    'Kings': 'Sacramento Kings',
    'Sacramento Kings': 'Sacramento Kings',
    'Kansas City Kings': 'Sacramento Kings',
    'Cincinnati Royals': 'Sacramento Kings',
    'Rochester Royals': 'Sacramento Kings',
    
    # Philadelphia 76ers and historical names
    'Philadelphia': 'Philadelphia 76ers',
    'PHI': 'Philadelphia 76ers',
    '76ers': 'Philadelphia 76ers',
    'Sixers': 'Philadelphia 76ers',
    'Philadelphia 76ers': 'Philadelphia 76ers',
    'Syracuse Nationals': 'Philadelphia 76ers',
    
    # Los Angeles Lakers and historical names
    'LA Lakers': 'Los Angeles Lakers',
    'LAL': 'Los Angeles Lakers',
    'Lakers': 'Los Angeles Lakers',
    'Los Angeles Lakers': 'Los Angeles Lakers',
    'Minneapolis Lakers': 'Los Angeles Lakers',
    
    # Houston Rockets and historical names
    'Houston': 'Houston Rockets',
    'HOU': 'Houston Rockets',
    'Rockets': 'Houston Rockets',
    'Houston Rockets': 'Houston Rockets',
    'San Diego Rockets': 'Houston Rockets',
    
    # Oklahoma City Thunder and historical names
    'Oklahoma City': 'Oklahoma City Thunder',
    'OKC': 'Oklahoma City Thunder',
    'Thunder': 'Oklahoma City Thunder',
    'Oklahoma City Thunder': 'Oklahoma City Thunder',
    'Seattle SuperSonics': 'Oklahoma City Thunder',
    'Sonics': 'Oklahoma City Thunder',
    'SEA': 'Oklahoma City Thunder',
    
    # Memphis Grizzlies and historical names
    'Memphis': 'Memphis Grizzlies',
    'MEM': 'Memphis Grizzlies',
    'Grizzlies': 'Memphis Grizzlies',
    'Memphis Grizzlies': 'Memphis Grizzlies',
    'Vancouver Grizzlies': 'Memphis Grizzlies',
    
    # New Orleans Pelicans and historical names
    'New Orleans': 'New Orleans Pelicans',
    'NOP': 'New Orleans Pelicans',
    'Pelicans': 'New Orleans Pelicans',
    'New Orleans Pelicans': 'New Orleans Pelicans',
    'New Orleans Hornets': 'New Orleans Pelicans',
    'New Orleans/Oklahoma City Hornets': 'New Orleans Pelicans',
    'NOK': 'New Orleans Pelicans',
    'NOH': 'New Orleans Pelicans',
    
    # Utah Jazz and historical names
    'Utah': 'Utah Jazz',
    'UTA': 'Utah Jazz',
    'Jazz': 'Utah Jazz',
    'Utah Jazz': 'Utah Jazz',
    'New Orleans Jazz': 'Utah Jazz',
    
    # Charlotte Hornets and historical names
    'Charlotte': 'Charlotte Hornets',
    'CHA': 'Charlotte Hornets',
    'Hornets': 'Charlotte Hornets',
    'Charlotte Hornets': 'Charlotte Hornets',
    'Charlotte Bobcats': 'Charlotte Hornets',
    'Bobcats': 'Charlotte Hornets',
    'CHO': 'Charlotte Hornets',
    
    # Brooklyn Nets and historical names
    'Brooklyn': 'Brooklyn Nets',
    'BKN': 'Brooklyn Nets',
    'Nets': 'Brooklyn Nets',
    'Brooklyn Nets': 'Brooklyn Nets',
    'New Jersey Nets': 'Brooklyn Nets',
    'NJN': 'Brooklyn Nets',
    'BRK': 'Brooklyn Nets',
    
    # Golden State Warriors and historical names
    'Golden State': 'Golden State Warriors',
    'GSW': 'Golden State Warriors',
    'Warriors': 'Golden State Warriors',
    'Golden State Warriors': 'Golden State Warriors',
    'San Francisco Warriors': 'Golden State Warriors',
    
    # Phoenix Suns
    'Phoenix': 'Phoenix Suns',
    'PHX': 'Phoenix Suns',
    'PHO': 'Phoenix Suns',
    'Suns': 'Phoenix Suns',
    'Phoenix Suns': 'Phoenix Suns',
    
    # Portland Trail Blazers
    'Portland': 'Portland Trail Blazers',
    'POR': 'Portland Trail Blazers',
    'Blazers': 'Portland Trail Blazers',
    'Trail Blazers': 'Portland Trail Blazers',
    'Portland Trail Blazers': 'Portland Trail Blazers',
    
    # San Antonio Spurs
    'San Antonio': 'San Antonio Spurs',
    'SAS': 'San Antonio Spurs',
    'Spurs': 'San Antonio Spurs',
    'San Antonio Spurs': 'San Antonio Spurs',
    
    # Toronto Raptors
    'Toronto': 'Toronto Raptors',
    'TOR': 'Toronto Raptors',
    'Raptors': 'Toronto Raptors',
    'Toronto Raptors': 'Toronto Raptors',
    
    # Milwaukee Bucks
    'Milwaukee': 'Milwaukee Bucks',
    'MIL': 'Milwaukee Bucks',
    'Bucks': 'Milwaukee Bucks',
    'Milwaukee Bucks': 'Milwaukee Bucks',
    
    # Minnesota Timberwolves
    'Minnesota': 'Minnesota Timberwolves',
    'MIN': 'Minnesota Timberwolves',
    'Timberwolves': 'Minnesota Timberwolves',
    'Wolves': 'Minnesota Timberwolves',
    'Minnesota Timberwolves': 'Minnesota Timberwolves',
    
    # Denver Nuggets
    'Denver': 'Denver Nuggets',
    'DEN': 'Denver Nuggets',
    'Nuggets': 'Denver Nuggets',
    'Denver Nuggets': 'Denver Nuggets',
    
    # Miami Heat
    'Miami': 'Miami Heat',
    'MIA': 'Miami Heat',
    'Heat': 'Miami Heat',
    'Miami Heat': 'Miami Heat',
    
    # Cleveland Cavaliers
    'Cleveland': 'Cleveland Cavaliers',
    'CLE': 'Cleveland Cavaliers',
    'Cavaliers': 'Cleveland Cavaliers',
    'Cavs': 'Cleveland Cavaliers',
    'Cleveland Cavaliers': 'Cleveland Cavaliers',
    
    # Boston Celtics
    'Boston': 'Boston Celtics',
    'BOS': 'Boston Celtics',
    'Celtics': 'Boston Celtics',
    'Boston Celtics': 'Boston Celtics',
    
    # Detroit Pistons
    'Detroit': 'Detroit Pistons',
    'DET': 'Detroit Pistons',
    'Pistons': 'Detroit Pistons',
    'Detroit Pistons': 'Detroit Pistons',
    
    # Indiana Pacers
    'Indiana': 'Indiana Pacers',
    'IND': 'Indiana Pacers',
    'Pacers': 'Indiana Pacers',
    'Indiana Pacers': 'Indiana Pacers',
    
    # Chicago Bulls
    'Chicago': 'Chicago Bulls',
    'CHI': 'Chicago Bulls',
    'Bulls': 'Chicago Bulls',
    'Chicago Bulls': 'Chicago Bulls',
    
    # Dallas Mavericks
    'Dallas': 'Dallas Mavericks',
    'DAL': 'Dallas Mavericks',
    'Mavericks': 'Dallas Mavericks',
    'Mavs': 'Dallas Mavericks',
    'Dallas Mavericks': 'Dallas Mavericks',
    
    # Orlando Magic
    'Orlando': 'Orlando Magic',
    'ORL': 'Orlando Magic',
    'Magic': 'Orlando Magic',
    'Orlando Magic': 'Orlando Magic',
    
    # New York Knicks
    'New York': 'New York Knicks',
    'NYK': 'New York Knicks',
    'Knicks': 'New York Knicks',
    'New York Knicks': 'New York Knicks'
}

# Add full names as keys (though they're already included above)
for team in set(team_mapping.values()):
    team_mapping[team] = team

print(f"Created team mapping with {len(team_mapping)} entries")
print(f"Standardized to {len(set(team_mapping.values()))} unique team names")

Created team mapping with 159 entries
Standardized to 30 unique team names


In [97]:
def standardize_team_names(df):
    df_std = df.copy()
    
    # Find team name columns
    team_cols = [col for col in df.columns if 'team' in col.lower()]
    
    # Apply mapping
    for col in team_cols:
        df_std[col] = df_std[col].map(lambda x: team_mapping.get(str(x), x))
    
    return df_std

# Standardize team names
shots_std = standardize_team_names(shots)
team_stats_std = standardize_team_names(team_stats)
player_data_std = standardize_team_names(player_data)

# Remove League Average entries from team_stats
team_stats_std = team_stats_std[team_stats_std['team'] != 'League Average']
print(f"Removed {len(team_stats) - len(team_stats_std)} 'League Average' entries")

player_data_std = player_data_std[player_data_std['team'] != 'TOT']
print(f"Removed {len(player_data) - len(player_data_std)} 'TOT' entries from player data")

# Check results
for df_name, df in [('shots', shots_std), ('team_stats', team_stats_std), ('player_data', player_data_std)]:
    team_cols = [col for col in df.columns if 'team' in col.lower()]
    for col in team_cols:
        print(f"{df_name}.{col}: {df[col].nunique()} unique values")

Removed 88 'League Average' entries
Removed 2962 'TOT' entries from player data
shots.TEAM_ID: 30 unique values
shots.TEAM_NAME: 30 unique values
shots.HOME_TEAM: 30 unique values
shots.AWAY_TEAM: 30 unique values
team_stats.team: 72 unique values
player_data.team: 99 unique values


## Conference Mapping

In [98]:
eastern_conf_teams = [
    'Atlanta Hawks',
    'Boston Celtics',
    'Brooklyn Nets',
    'Charlotte Hornets',
    'Chicago Bulls',
    'Cleveland Cavaliers',
    'Detroit Pistons',
    'Indiana Pacers',
    'Miami Heat',
    'Milwaukee Bucks',
    'New York Knicks',
    'Orlando Magic',
    'Philadelphia 76ers',
    'Toronto Raptors',
    'Washington Wizards',

    'New York Nets',  # ABA Eastern Division
    'Kentucky Colonels',  # ABA Eastern Division
    'Spirits of St. Louis',  # ABA Eastern Division
    'Virginia Squires',  # ABA Eastern Division
    'Carolina Cougars',  # ABA Eastern Division
    'Memphis Tams',  # ABA Eastern Division
    'The Floridians',  # ABA Eastern Division
    'Memphis Pros',  # ABA Eastern Division
    'Pittsburgh Condors',  # ABA Eastern Division
    'Miami Floridians',  # ABA Eastern Division
    'Pittsburgh Pipers',  # ABA Eastern Division
    'Washington Capitols',  # BAA/NBA Eastern Division
    'Minnesota Pipers',  # ABA Eastern Division
    'New Jersey Americans',  # ABA Eastern Division
    'Philadelphia Warriors',  # NBA Eastern Division
    'Fort Wayne Pistons',  # NBA Eastern Division (before moving to Detroit)
    'Indianapolis Olympians',  # NBA Western Division (but geographically eastern)
    'Anderson Packers',  # NBA Eastern Division
    'Chicago Stags',  # BAA/NBA Eastern Division
    'Sheboygan Red Skins',  # NBA Western Division (but later Eastern)
    'Indianapolis Jets',  # BAA Eastern Division
    'Providence Steamrollers',  # BAA Eastern Division
    'Cleveland Rebels',  # BAA Eastern Division
    'Detroit Falcons',  # BAA Eastern Division
    'Pittsburgh Ironmen',  # BAA Eastern Division
    'Toronto Huskies',  # BAA Eastern Division

    'ATL',  # Atlanta Hawks
    'BOS',  # Boston Celtics
    'BKN',  # Brooklyn Nets
    'CHA',  # Charlotte Hornets
    'CHI',  # Chicago Bulls
    'CLE',  # Cleveland Cavaliers
    'DET',  # Detroit Pistons
    'IND',  # Indiana Pacers
    'MIA',  # Miami Heat
    'MIL',  # Milwaukee Bucks
    'NYK',  # New York Knicks
    'ORL',  # Orlando Magic
    'PHI',  # Philadelphia 76ers
    'TOR',  # Toronto Raptors
    'WAS',  # Washington Wizards
    
    # Historical Eastern Conference teams
    'CHH',  # Charlotte Hornets (original)
    'WSB',  # Washington Bullets
    'BUF',  # Buffalo Braves
    'NYN',  # New York Nets (ABA)
    'NYA',  # New York Americans
    'KEN',  # Kentucky Colonels (ABA)
    'SSL',  # Spirits of St. Louis (ABA)
    'INA',  # Indiana Pacers (ABA)
    'VIR',  # Virginia Squires (ABA)
    'CAP',  # Capital Bullets
    'CAR',  # Carolina Cougars (ABA)
    'BAL',  # Baltimore Bullets
    'FLO',  # The Floridians (ABA)
    'PTC',  # Pittsburgh Condors (ABA)
    'CIN',  # Cincinnati Royals
    'MMF',  # Miami Floridians (ABA)
    'PTP',  # Pittsburgh Pipers (ABA)
    'WSA',  # Washington Capitols
    'MNP',  # Minnesota Pipers (ABA)
    'NJA',  # New Jersey Americans (ABA)
    'SYR',  # Syracuse Nationals
    'CHZ',  # Chicago Zephyrs
    'PHW',  # Philadelphia Warriors
    'CHP',  # Chicago Packers
    'FTW',  # Fort Wayne Pistons
    'ROC',  # Rochester Royals
    'BLB',  # Baltimore Bullets (original)
    'INO',  # Indianapolis Olympians
    'WSC',  # Washington Capitols
    'CHS',  # Chicago Stags
    'AND',  # Anderson Packers
    'INJ',  # Indianapolis Jets
    'PRO',  # Providence Steamrollers
    'DTF',  # Detroit Falcons
    'CLR',  # Cleveland Rebels
    'TRH',  # Toronto Huskies
    'PIT',  # Pittsburgh Ironmen
]

western_conf_teams = [
    'Dallas Mavericks',
    'Denver Nuggets',
    'Golden State Warriors',
    'Houston Rockets',
    'Los Angeles Clippers',
    'Los Angeles Lakers',
    'Memphis Grizzlies',
    'Minnesota Timberwolves',
    'New Orleans Pelicans',
    'Oklahoma City Thunder',
    'Phoenix Suns',
    'Portland Trail Blazers',
    'Sacramento Kings',
    'San Antonio Spurs',
    'Utah Jazz',

    'San Diego Sails',  # ABA Western Division
    'Utah Stars',  # ABA Western Division
    'Kansas City-Omaha Kings',  # NBA Midwest Division (Western Conference)
    'Memphis Sounds',  # ABA Western Division
    'San Diego Conquistadors',  # ABA Western Division
    'Denver Rockets',  # ABA Western Division
    'Dallas Chaparrals',  # ABA Western Division
    'Texas Chaparrals',  # ABA Western Division
    'Los Angeles Stars',  # ABA Western Division
    'New Orleans Buccaneers',  # ABA Western Division
    'Houston Mavericks',  # ABA Western Division
    'Oakland Oaks',  # ABA Western Division
    'Anaheim Amigos',  # ABA Western Division
    'Minnesota Muskies',  # ABA Western Division
    'St. Louis Bombers',  # BAA/NBA Western Division
    'Waterloo Hawks',  # NBA Western Division

    'DAL',  # Dallas Mavericks
    'DEN',  # Denver Nuggets
    'GSW',  # Golden State Warriors
    'HOU',  # Houston Rockets
    'LAC',  # Los Angeles Clippers
    'LAL',  # Los Angeles Lakers
    'MEM',  # Memphis Grizzlies
    'MIN',  # Minnesota Timberwolves
    'NOP',  # New Orleans Pelicans
    'OKC',  # Oklahoma City Thunder
    'PHX',  # Phoenix Suns
    'POR',  # Portland Trail Blazers
    'SAC',  # Sacramento Kings
    'SAS',  # San Antonio Spurs
    'UTA',  # Utah Jazz
    
    # Historical Western Conference teams
    'VAN',  # Vancouver Grizzlies
    'KCK',  # Kansas City Kings
    'SDC',  # San Diego Clippers
    'NOJ',  # New Orleans Jazz
    'UTS',  # Utah Stars (ABA)
    'SAA',  # San Antonio Spurs (ABA)
    'SDS',  # San Diego Sails (ABA)
    'DNA',  # Denver Nuggets (ABA)
    'SDA',  # San Diego Conquistadors (ABA)
    'MMS',  # Memphis Sounds (ABA)
    'KCO',  # Kansas City-Omaha Kings
    'DNR',  # Denver Rockets (ABA)
    'MMT',  # Memphis Tams (ABA)
    'DLC',  # Dallas Chaparrals (ABA)
    'MMP',  # Memphis Pros (ABA)
    'SFW',  # San Francisco Warriors
    'SDR',  # San Diego Rockets
    'TEX',  # Texas Chaparrals (ABA)
    'LAS',  # Los Angeles Stars (ABA)
    'NOB',  # New Orleans Buccaneers (ABA)
    'OAK',  # Oakland Oaks (ABA)
    'HSM',  # Houston Mavericks (ABA)
    'ANA',  # Anaheim Amigos (ABA)
    'STL',  # St. Louis Hawks/Bombers
    'MNM',  # Minnesota Muskies (ABA)
    'MNL',  # Minneapolis Lakers
    'MLH',  # Milwaukee Hawks
    'TRI',  # Tri-Cities Blackhawks
    'DNN',  # Denver Nuggets (original)
    'WAT',  # Waterloo Hawks
    'STB',  # St. Louis Bombers
    'SHE',  # Sheboygan Red Skins
]

def add_conference_mappings(df, team_col=None):
    df_conf = df.copy()
    
    # Convert conference team lists to uppercase for case-insensitive comparison
    eastern_conf_upper = [team.upper() for team in eastern_conf_teams]
    western_conf_upper = [team.upper() for team in western_conf_teams]
    
    # Find team name columns if not specified
    if team_col is None:
        team_cols = [col for col in df.columns if 'team' in col.lower()]
        if len(team_cols) > 0:
            team_col = team_cols[0]  # Use the first team column found
        else:
            print("No team column found")
            return df_conf
    
    # Make sure the column exists
    if team_col in df.columns:
        # Add conference column
        df_conf['conference'] = df_conf[team_col].apply(
            lambda x: 'EAST' if str(x).upper() in eastern_conf_upper 
                    else 'WEST' if str(x).upper() in western_conf_upper 
                    else 'Unknown'
        )
        
        # Print conference distribution
        print(f"Conference distribution:")
        print(df_conf['conference'].value_counts())
    
    return df_conf

# Add conference information
shots_std = add_conference_mappings(shots_std, 'TEAM_NAME')
team_stats_std = add_conference_mappings(team_stats_std, 'team')
player_data_std = add_conference_mappings(player_data_std, 'team')  # Add player data with 'team' column

# Check results
for df_name, df in [('shots', shots_std), ('team_stats', team_stats_std), ('player_data', player_data_std)]:
    if 'conference' in df.columns:
        print(f"{df_name} conference distribution:")
        print(df['conference'].value_counts())

Conference distribution:
conference
WEST    2128199
EAST    2102871
Name: count, dtype: int64
Conference distribution:
conference
EAST    959
WEST    829
Name: count, dtype: int64
Conference distribution:
conference
EAST    15975
WEST    13601
Name: count, dtype: int64
shots conference distribution:
conference
WEST    2128199
EAST    2102871
Name: count, dtype: int64
team_stats conference distribution:
conference
EAST    959
WEST    829
Name: count, dtype: int64
player_data conference distribution:
conference
EAST    15975
WEST    13601
Name: count, dtype: int64


## Percentage Conversion

In [99]:
def convert_percentages(df):
    df_pct = df.copy()
    
    # Find percentage columns - add 'percent' to the search terms
    pct_cols = [col for col in df.columns if any(x in col.lower() for x in ['percentage', 'pct', 'percent'])]
    
    if len(pct_cols) > 0:
        print(f"Found {len(pct_cols)} percentage columns: {pct_cols}")
        
        # Convert each percentage column
        for col in pct_cols:
            if df_pct[col].dtype == 'object':  # Only convert string columns
                # Remove % sign and convert to float, then divide by 100
                df_pct[col] = df_pct[col].str.rstrip('%').astype('float') / 100.0
                print(f"Converted {col} to decimal values")
    else:
        print("No percentage columns found")
    
    return df_pct

In [100]:
team_stats_std = convert_percentages(team_stats_std)
player_data_std = convert_percentages(player_data_std)  # Add player data conversion

for df_name, df in [('shots', shots_std), ('team_stats', team_stats_std), ('player_data', player_data_std)]:
    pct_cols = [col for col in df.columns if any(x in col.lower() for x in ['percentage', 'pct'])]
    for col in pct_cols:
        if col in df.columns:
            print(f"{df_name}.{col} statistics:")
            print(df[col].describe())

Found 4 percentage columns: ['fg_percent', 'x3p_percent', 'x2p_percent', 'ft_percent']
Found 5 percentage columns: ['fg_percent', 'x3p_percent', 'x2p_percent', 'e_fg_percent', 'ft_percent']


Numeric Column Handling

In [101]:
def handle_numeric_columns(df):
    df_num = df.copy()
    
    # Define patterns for columns that are likely not numeric
    exclude_patterns = [
        # Shot data columns
        'name', 'team', 'position', 'date', 'season', 'location', 
        'event_type', 'action_type', 'shot_type', 'basic_zone', 
        'zone_abb', 'zone_range', 'zone_name', 'home_team', 'away_team',
        'position_group', 'season_1', 'season_2',  # Added specific season columns
        
        # Team stats columns
        'abbreviation', 'player', 'pos', 'lg', 'tm', 'experience',
        'birth_year', 'birth_date', 'college', 'slug', 'arena',
        'playoffs', 'winner', 'replaced', 'type', 'number_tm',
        'hof', 'from', 'to', 'ht_in_in', 'wt',
        
        # Player data columns
        'seas_id', 'player_id', 'player', 'pos', 'lg', 'conference'
        # Note: 'season', 'birth_year', 'age', 'experience' are already in the list above
    ]
    # Filter columns that are likely to be numeric
    numeric_candidates = [col for col in df.columns 
                         if not any(pattern in col.lower() for pattern in exclude_patterns)]
    
    if len(numeric_candidates) > 0:
        print(f"Found {len(numeric_candidates)} potential numeric columns")
        
        # Convert to numeric and handle missing values
        for col in numeric_candidates:
            try:
                # Convert to numeric first
                df_num[col] = pd.to_numeric(df_num[col], errors='coerce')
                
                # Handle missing values
                if df_num[col].isna().all():
                    # If all values are NaN, set the entire column to 0
                    df_num[col] = 0
                    print(f"Column {col}: All values were NaN, set to 0")
                elif df_num[col].isna().any():
                    # If some values are NaN, fill them with 0
                    missing_count = df_num[col].isna().sum()
                    df_num[col] = df_num[col].fillna(0)
                    print(f"Column {col}: Filled {missing_count} missing values with 0")
                else:
                    print(f"Column {col}: Successfully converted to numeric with no missing values")
            except Exception as e:
                print(f"Warning: Could not convert column {col} to numeric: {str(e)}")
    else:
        print("No potential numeric columns found")
    
    return df_num

In [102]:
shots_std = handle_numeric_columns(shots_std)
team_stats_std = handle_numeric_columns(team_stats_std)
player_data_std = handle_numeric_columns(player_data_std)  # Add player data handling

# Check results
for df_name, df in [('shots', shots_std), ('team_stats', team_stats_std), ('player_data', player_data_std)]:
    numeric_cols = df.select_dtypes(include=['number']).columns
    print(f"\n{df_name} has {len(numeric_cols)} numeric columns")
    if len(numeric_cols) > 0:
        print(f"Sample numeric columns: {list(numeric_cols)[:5]}")

Found 8 potential numeric columns
Column GAME_ID: Successfully converted to numeric with no missing values
Column SHOT_MADE: Successfully converted to numeric with no missing values
Column LOC_X: Successfully converted to numeric with no missing values
Column LOC_Y: Successfully converted to numeric with no missing values
Column SHOT_DISTANCE: Successfully converted to numeric with no missing values
Column QUARTER: Successfully converted to numeric with no missing values
Column MINS_LEFT: Successfully converted to numeric with no missing values
Column SECS_LEFT: Successfully converted to numeric with no missing values
Found 22 potential numeric columns
Column g: Successfully converted to numeric with no missing values
Column mp_per_game: Filled 172 missing values with 0
Column fg_per_game: Successfully converted to numeric with no missing values
Column fga_per_game: Successfully converted to numeric with no missing values
Column fg_percent: Successfully converted to numeric with no mis

## Coordinate System Standardization

In [103]:
# Standardize court coordinates
def standardize_coordinates(df):
    df_std = df.copy()
    
    # Check if coordinate columns exist
    if 'LOC_X' in df.columns and 'LOC_Y' in df.columns:
        # Rename to lowercase
        df_std.rename(columns={'LOC_X': 'loc_x', 'LOC_Y': 'loc_y'}, inplace=True)
        
        # Calculate shot distance and angle if not present
        if 'shot_distance' not in df_std.columns:
            df_std['shot_distance'] = np.sqrt(df_std['loc_x']**2 + df_std['loc_y']**2) / 10  # Convert to feet
        
        if 'shot_angle' not in df_std.columns:
            df_std['shot_angle'] = np.arctan2(df_std['loc_y'], df_std['loc_x']) * 180 / np.pi  # Convert to degrees
    
    return df_std

In [104]:
shots_std = standardize_coordinates(shots_std)

# Check results
if 'loc_x' in shots_std.columns:
    print("Coordinate statistics:")
    print(shots_std[['loc_x', 'loc_y', 'shot_distance', 'shot_angle']].describe())

Coordinate statistics:
              loc_x         loc_y  shot_distance    shot_angle
count  4.231070e+06  4.231070e+06   4.231070e+06  4.231070e+06
mean   9.484333e-02  1.239039e+01   1.544464e+00  8.981937e+01
std    1.026559e+01  8.554561e+00   9.672471e-01  3.264137e+01
min   -2.500000e+01  5.000000e-02   1.500000e-02  2.075925e-01
25%   -2.900000e+00  5.875000e+00   6.347277e-01  7.384607e+01
50%   -0.000000e+00  8.050000e+00   1.223162e+00  9.000000e+01
75%    2.900000e+00  1.875000e+01   2.402712e+00  1.065430e+02
max    2.500000e+01  9.365000e+01   9.507514e+00  1.797954e+02


## Player Name Standardization

In [105]:
# Standardize player names
def standardize_player_names(df, name_cols=None):
    df_std = df.copy()
    
    # Find player name columns if not specified
    if name_cols is None:
        name_cols = [col for col in df.columns if 'player' in col.lower() and 'name' in col.lower()]
    
    if len(name_cols) > 0:
        print(f"Found {len(name_cols)} player name columns: {name_cols}")
        
        # Standardize format for each column
        for col in name_cols:
            if col in df.columns:
                # Check if column has values before processing
                non_null_count = df_std[col].notna().sum()
                print(f"Column {col} has {non_null_count} non-null values out of {len(df_std)} records")
                
                if non_null_count > 0:
                    # First, convert NaN values to empty strings
                    df_std[col] = df_std[col].fillna('')
                    
                    # Convert all values to strings before applying string methods
                    df_std[col] = df_std[col].astype(str)
                    
                    # Option 1: Convert to UPPERCASE (NBA data cleaner approach)
                    df_std[col] = df_std[col].str.strip().str.upper()
                    
                    # Option 2: Convert to Title Case (current notebook approach)
                    # df_std[col] = df_std[col].str.title().str.strip()
                    
                    # Convert empty strings back to NaN
                    df_std[col] = df_std[col].replace('', pd.NA).infer_objects(copy=False)
                    
                    # Show sample of standardized names
                    unique_names = df_std[col].dropna().unique()
                    sample_size = min(5, len(unique_names))
                    if sample_size > 0:
                        print(f"Sample standardized names: {list(unique_names[:sample_size])}")
                    
                    print(f"After standardization, column {col} has {df_std[col].nunique()} unique values")
    else:
        print("No player name columns found")
    
    return df_std

In [106]:
def standardize_player_names(df, name_cols=None):
    df_std = df.copy()
    
    # Find player name columns if not specified
    if name_cols is None:
        name_cols = [col for col in df.columns if 'player' in col.lower() and 'name' in col.lower()]
    
    if len(name_cols) > 0:
        print(f"Found {len(name_cols)} player name columns: {name_cols}")
        
        # Standardize format for each column
        for col in name_cols:
            if col in df.columns:
                # Check if column has values before processing
                non_null_count = df_std[col].notna().sum()
                print(f"Column {col} has {non_null_count} non-null values out of {len(df_std)} records")
                
                if non_null_count > 0:
                    # First, convert NaN values to empty strings
                    df_std[col] = df_std[col].fillna('')
                    
                    # Convert all values to strings before applying string methods
                    df_std[col] = df_std[col].astype(str)
                    
                    df_std[col] = df_std[col].str.strip().str.upper()
                    
                    # Convert empty strings back to NaN
                    df_std[col] = df_std[col].replace('', pd.NA).infer_objects(copy=False)
                    
                    # Show sample of standardized names
                    unique_names = df_std[col].dropna().unique()
                    sample_size = min(5, len(unique_names))
                    if sample_size > 0:
                        print(f"Sample standardized names: {list(unique_names[:sample_size])}")
                    
                    print(f"After standardization, column {col} has {df_std[col].nunique()} unique values")
    else:
        print("No player name columns found")
    
    return df_std

In [107]:
# Standardize player names
shots_std = standardize_player_names(shots_std)
player_data_std = standardize_player_names(player_data_std, ['player'])  # Add player data with 'player' column

# Check results
for df_name, df, col_name in [('shots', shots_std, 'PLAYER_NAME'), ('player_data', player_data_std, 'player')]:
    if col_name in df.columns:
        print(f"{df_name}.{col_name}: {df[col_name].nunique()} unique values")

Found 1 player name columns: ['PLAYER_NAME']
Column PLAYER_NAME has 4231070 non-null values out of 4231070 records
Sample standardized names: ['ANTHONY MORROW', 'KELENNA AZUBUIKE', 'GRANT HILL', 'DANIEL GIBSON', 'C.J. WATSON']
After standardization, column PLAYER_NAME has 2164 unique values
Found 1 player name columns: ['player']
Column player has 29576 non-null values out of 29576 records
Sample standardized names: ['A.J. GREEN', 'A.J. LAWSON', 'AJ JOHNSON', 'AARON GORDON', 'AARON HOLIDAY']
After standardization, column player has 5252 unique values
shots.PLAYER_NAME: 2164 unique values
player_data.player: 5252 unique values


## Temporal Standardization

In [108]:
def handle_dates(df, date_cols=None):
    df_dates = df.copy()
    
    # Find date columns if not specified
    if date_cols is None:
        date_cols = [col for col in df.columns if 'date' in col.lower()]
    
    if len(date_cols) > 0:
        print(f"Found {len(date_cols)} date columns: {date_cols}")
        
        # Convert each date column to datetime
        for col in date_cols:
            if col in df.columns:
                try:
                    # Store original values to check conversion success
                    original_values = df_dates[col].copy()
                    
                    # Convert to datetime with coercion for invalid dates
                    df_dates[col] = pd.to_datetime(df_dates[col], errors='coerce')
                    
                    # Check conversion success
                    success_count = df_dates[col].notna().sum()
                    total_count = len(df_dates[col])
                    success_rate = success_count / total_count * 100 if total_count > 0 else 0
                    
                    print(f"Column {col}: Converted {success_count}/{total_count} values to datetime ({success_rate:.1f}%)")
                    
                    # Show sample of before and after for verification
                    if success_count > 0:
                        sample_idx = df_dates[col].first_valid_index()
                        if sample_idx is not None:
                            original_sample = original_values.iloc[sample_idx]
                            converted_sample = df_dates[col].iloc[sample_idx]
                            print(f"  Sample conversion: '{original_sample}' → {converted_sample}")
                except Exception as e:
                    print(f"Error converting {col} to datetime: {str(e)}")
    else:
        print("No date columns found")
    
    # Add season column if not present
    if 'season' not in df_dates.columns and 'GAME_DATE' in df_dates.columns:
        # Extract season (assuming season starts in October and ends in June)
        def get_season(date):
            if pd.isna(date):
                return None
            year = date.year
            month = date.month
            if month >= 10:  # October to December
                return f"{year}-{year+1}"
            else:  # January to June
                return f"{year-1}-{year}"
        
        df_dates['season'] = df_dates['GAME_DATE'].apply(get_season)
        print("Added 'season' column based on GAME_DATE")
        
        # Show season distribution
        season_counts = df_dates['season'].value_counts()
        print("Season distribution:")
        print(season_counts.head())
    
    return df_dates

In [109]:
shots_std = handle_dates(shots_std)
team_stats_std = handle_dates(team_stats_std)
player_data_std = handle_dates(player_data_std)  # Add player data handling

# Check results
for df_name, df in [('shots', shots_std), ('team_stats', team_stats_std), ('player_data', player_data_std)]:
    date_cols = [col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])]
    if len(date_cols) > 0:
        print(f"\n{df_name} date columns: {date_cols}")
        for col in date_cols:
            print(f"{df_name}.{col} range: {df[col].min()} to {df[col].max()}")

Found 1 date columns: ['GAME_DATE']
Column GAME_DATE: Converted 4231070/4231070 values to datetime (100.0%)
  Sample conversion: '04-15-2009' → 2009-04-15 00:00:00
No date columns found
No date columns found

shots date columns: ['GAME_DATE']
shots.GAME_DATE range: 2003-10-28 00:00:00 to 2024-04-14 00:00:00


## Update Column Names

In [110]:
if 'PLAYER_NAME' in shots_std.columns and 'player_name' not in shots_std.columns:
    shots_std.rename(columns={'PLAYER_NAME': 'player_name'}, inplace=True)
    print("Renamed 'PLAYER_NAME' to 'player_name' in shots data")

if 'TEAM_NAME' in shots_std.columns and 'team_name' not in shots_std.columns:
    shots_std.rename(columns={'TEAM_NAME': 'team_name'}, inplace=True)
    print("Renamed 'TEAM_NAME' to 'team_name' in shots data")

player_column_mapping = {
    'pts': 'points',
    'fga': 'field_goal_attempts',
    'fta': 'free_throw_attempts',
    'tov': 'turnovers',
    'mp': 'minutes'
}

for old_name, new_name in player_column_mapping.items():
    if old_name in player_data_std.columns and new_name not in player_data_std.columns:
        player_data_std.rename(columns={old_name: new_name}, inplace=True)
        print(f"Renamed '{old_name}' to '{new_name}' in player data")

Renamed 'PLAYER_NAME' to 'player_name' in shots data
Renamed 'TEAM_NAME' to 'team_name' in shots data
Renamed 'pts' to 'points' in player data
Renamed 'fga' to 'field_goal_attempts' in player data
Renamed 'fta' to 'free_throw_attempts' in player data
Renamed 'tov' to 'turnovers' in player data
Renamed 'mp' to 'minutes' in player data


## Save Standardized Data

In [111]:
shots_std.to_csv(processed_dir / 'standardized_shots.csv', index=False)
team_stats_std.to_csv(processed_dir / 'standardized_team.csv', index=False)
player_data_std.to_csv(processed_dir / 'standardized_player.csv', index=False)

print(f"Saved standardized shot data to {processed_dir / 'standardized_shots.csv'}")
print(f"Saved standardized team stats to {processed_dir / 'standardized_team.csv'}")
print(f"Saved standardized player data to {processed_dir / 'standardized_player.csv'}")

Saved standardized shot data to ../data/processed/standardized_shots.csv
Saved standardized team stats to ../data/processed/standardized_team.csv
Saved standardized player data to ../data/processed/standardized_player.csv


## Data Standardization Summary

We've standardized the following aspects of our data:

1. **Player Names**: Populated missing player names from player_id and standardized to consistent UPPERCASE format for reliable matching and analysis
2. **Team Names**: Unified team names across all datasets, handling variations and historical changes including historical franchise relocations
3. **Conference Mappings**: Added Eastern and Western conference designations to each team, enabling conference-based analysis and comparisons
4. **Percentage Values**: Converted percentage strings to decimal values for consistent numerical analysis
5. **Numeric Columns**: Identified and converted potential numeric columns, handling missing values appropriately
6. **Coordinates**: Standardized court coordinates and calculated derived spatial features like shot distance and angle
7. **Temporal Information**: Standardized dates across all datasets, added season information, and ensured proper datetime formatting for time-based analysis

These comprehensive standardization steps ensure that our data is consistent, clean, and properly formatted across all datasets, enabling accurate analysis and modeling in subsequent notebooks. The standardization process follows best practices from the NBA data cleaner class, ensuring compatibility with other components of the project.

Next: Feature engineering to create additional meaningful features